As we have seen in previous notebook, more feature engineering is required. Few of the Kaggle kernels used moving averages, quantiles and trends.
<ol>
    <li>Quantile features from this kernel: https://www.kaggle.com/andrekos/basic-feature-benchmark-with-quantiles</li>
    <li>Trend features from this kernel: https://www.kaggle.com/jsaguiar/baseline-with-abs-and-trend-features</li>
    <li>Rolling features from this kernel: https://www.kaggle.com/wimwim/rolling-quantiles</li>
</ol>
I have incorporated above and updated the train_data_generator.py to generate these feature.

Train file is ready with train_data_generator.py and train_sampler.py. All we need to do now is to import and see.

In [4]:
import accoustic_sampler as acs
import data_formatter as dtFrm

sampler = acs.AccousticSampler('D:/PYTHON_WORKSPACES/Kaggles/EarthquakePrediction/LANL_Earthquake/data/train_data_new')
sampler.fit()
data_df = sampler.get()
# columns_to_drop=columns_to_drop
formatter = dtFrm.LANLDataFormatter(data_df=data_df, data_type='train', doTransform=True, doScale=True, cols_to_keep=50)
data_df = formatter.transform()
most_dependent_columns = formatter.getMostImpCols()


Data file already exists ..Fitting completed
Transformation completed ... 
Dropping columns  ['acc_mean', 'median_50', 'auc'] ...
Standard Scaler applied ... 


The variable 'most_dependent_columns' will be used for test data set as well. I have already trained the data set using LGBMRegressor and created a pickle file. All we need is to load the same. Please run the lgbm_test.py to generate the pickle model. The file also contains the CV test which are commented in order to improve the model if necessary.

In [5]:
#Using the entire data set for training
y_train = data_df['time_to_failure']
x_train_seg = data_df['segment_id']
x_train = data_df.drop(['time_to_failure','segment_id'], axis=1)
x_train.head(5)

,q05_roll_std_100,q01_roll_std_100,q05_roll_std_10,q01_roll_std_10,q05_roll_std_1000,q01_roll_std_1000,q01_log,q99_log,q99_roll_mean_10_log,min_roll_std_1000,...,max_roll_std_10_log,time_per_max_range,max_dist_from_mean,acc_max_log,chg_acc_min_log,chg_acc_max_log,ave_roll_std_1000,ave_roll_std_10,median_25,ave_roll_std_100
0,1.132463,1.084806,0.792875,0.931954,1.352746,1.829103,-0.828981,0.871222,0.796488,0.882394,...,0.620893,-0.602602,0.481358,0.474880,-0.615128,0.690355,0.397420,0.397058,-0.396808,0.337247
1,0.583159,0.443190,0.192855,0.302379,0.520259,0.692956,-0.828981,0.741979,0.886868,0.556486,...,-0.077081,-0.155580,0.036501,0.028676,0.108663,-0.003405,0.239752,0.191857,-0.396808,0.223631
2,-0.558992,-0.631891,-0.578320,-0.435531,-0.473787,-0.574577,0.991674,-1.199113,-1.201293,-0.165992,...,-1.193241,1.467542,-1.386264,-1.371912,1.038745,-1.343103,-0.577111,-0.537765,-0.396808,-0.524857
3,0.462218,0.351752,0.444617,0.302379,0.657004,0.752134,-0.157031,0.002353,-0.018339,0.335649,...,-1.048894,0.656167,-0.768290,-0.768394,1.193979,-1.096030,-0.097282,-0.085439,-0.396808,-0.076643
4,0.615791,0.699098,0.444617,0.931954,0.290687,0.417168,-0.459180,0.466500,0.405681,0.630478,...,-0.195788,0.204231,-0.124539,-0.128639,-0.025142,-0.338301,0.106920,0.139062,-0.396808,0.089279


In [6]:
import lightgbm as lgb
model = lgb.LGBMRegressor(bagging_fraction=0.5, bagging_freq=2, bagging_seed=5,
       boosting='gbdt', boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, feature_fraction=1.0, importance_type='split',
       learning_rate=0.001, max_depth=-1, metric='mae',
       min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=100,
       min_split_gain=0.0, n_estimators=20000, n_jobs=-1, num_leaves=128,
       objective='regression', random_state=None, reg_alpha=0.1,
       reg_lambda=1.0, silent=False, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbosity=-1)
model.fit(x_train, y_train, verbose=1000)

LGBMRegressor(bagging_fraction=0.5, bagging_freq=2, bagging_seed=5,
       boosting='gbdt', boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, feature_fraction=1.0, importance_type='split',
       learning_rate=0.001, max_depth=-1, metric='mae',
       min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=100,
       min_split_gain=0.0, n_estimators=20000, n_jobs=-1, num_leaves=128,
       objective='regression', random_state=None, reg_alpha=0.1,
       reg_lambda=1.0, silent=False, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbosity=-1)

In [7]:
#Let's load our test file which is again pre generated as test_data_model.csv.
import accoustic_sampler as acs

sampler = acs.AccousticSampler('D:/PYTHON_WORKSPACES/Kaggles/EarthquakePrediction/LANL_Earthquake/data/test_data',data_type='test')
sampler.fit()
test_data_df = sampler.get()

import data_formatter as dtFrm
formatter = dtFrm.LANLDataFormatter(data_df=test_data_df, data_type='test', doTransform=True, doScale=True,\
                                    most_dependent_columns=most_dependent_columns)
test_df = formatter.transform()
print('Test data size ',test_df.shape[0])
test_df.head(5)

Data file already exists ..Fitting completed
Transformation completed ... 
Dropping columns  ['acc_mean', 'median_50', 'auc'] ...
Standard Scaler applied ... 
Test data size  2624


,q05_roll_std_100,q01_roll_std_100,q05_roll_std_10,q01_roll_std_10,q05_roll_std_1000,q01_roll_std_1000,q01_log,q99_log,q99_roll_mean_10_log,min_roll_std_1000,...,time_per_max_range,max_dist_from_mean,acc_max_log,chg_acc_min_log,chg_acc_max_log,ave_roll_std_1000,ave_roll_std_10,median_25,ave_roll_std_100,segment_id
0,0.473070,0.578964,0.143651,0.199808,0.529071,0.512646,0.380333,-0.171774,-0.235943,-0.515151,...,0.444237,-0.241210,-0.243171,1.014980,-0.850613,-0.220017,-0.190820,0.400246,-0.182965,seg_00030f
1,0.031385,0.103841,-0.056244,-0.247975,0.077580,0.323945,-0.326368,0.156542,0.121378,0.273865,...,-0.465386,0.271509,0.263658,-0.258143,-0.275335,-0.065598,-0.068861,0.400246,-0.064598,seg_0012b5
2,0.031155,0.125786,-0.056244,-0.157804,-0.231023,-0.018358,-0.174712,0.156542,0.253136,-0.330997,...,-0.837817,1.150123,1.153133,-0.835552,0.427175,-0.027233,-0.072150,0.400246,-0.049649,seg_00184e
3,-1.037652,-0.846167,-1.081584,-0.980918,-0.958864,-0.966187,1.199520,-0.954900,-0.883336,-1.176640,...,0.620867,-0.800200,-0.792394,0.810318,-0.377557,-0.534459,-0.545865,2.446511,-0.517331,seg_003339
4,-0.368337,-0.378088,-0.308461,-0.247975,-0.630210,-0.580274,-0.008592,-0.003294,0.053253,-0.095551,...,-0.615400,0.547272,0.540320,-0.019989,0.495314,-0.153716,-0.166164,0.400246,-0.150474,seg_0042cc


In [8]:
x_test_seg = test_df['segment_id']
x_test = test_df.drop(['segment_id'],axis=1)
x_test.head(5)

,q05_roll_std_100,q01_roll_std_100,q05_roll_std_10,q01_roll_std_10,q05_roll_std_1000,q01_roll_std_1000,q01_log,q99_log,q99_roll_mean_10_log,min_roll_std_1000,...,max_roll_std_10_log,time_per_max_range,max_dist_from_mean,acc_max_log,chg_acc_min_log,chg_acc_max_log,ave_roll_std_1000,ave_roll_std_10,median_25,ave_roll_std_100
0,0.473070,0.578964,0.143651,0.199808,0.529071,0.512646,0.380333,-0.171774,-0.235943,-0.515151,...,-0.696964,0.444237,-0.241210,-0.243171,1.014980,-0.850613,-0.220017,-0.190820,0.400246,-0.182965
1,0.031385,0.103841,-0.056244,-0.247975,0.077580,0.323945,-0.326368,0.156542,0.121378,0.273865,...,0.317940,-0.465386,0.271509,0.263658,-0.258143,-0.275335,-0.065598,-0.068861,0.400246,-0.064598
2,0.031155,0.125786,-0.056244,-0.157804,-0.231023,-0.018358,-0.174712,0.156542,0.253136,-0.330997,...,0.976964,-0.837817,1.150123,1.153133,-0.835552,0.427175,-0.027233,-0.072150,0.400246,-0.049649
3,-1.037652,-0.846167,-1.081584,-0.980918,-0.958864,-0.966187,1.199520,-0.954900,-0.883336,-1.176640,...,-0.624776,0.620867,-0.800200,-0.792394,0.810318,-0.377557,-0.534459,-0.545865,2.446511,-0.517331
4,-0.368337,-0.378088,-0.308461,-0.247975,-0.630210,-0.580274,-0.008592,-0.003294,0.053253,-0.095551,...,0.349010,-0.615400,0.547272,0.540320,-0.019989,0.495314,-0.153716,-0.166164,0.400246,-0.150474


In [9]:
y_test_pred = model.predict(x_test)

In [10]:
import pandas as pd
data_dict = {'seg_id': x_test_seg.values,'time_to_failure': y_test_pred}
out_df = pd.DataFrame(data_dict)
print('Total predicted segments ',out_df.shape[0])
out_df.head(10)

Total predicted segments  2624


,seg_id,time_to_failure
0,seg_00030f,3.111684
1,seg_0012b5,5.850538
2,seg_00184e,6.642270
3,seg_003339,9.267204
4,seg_0042cc,8.048579
5,seg_004314,2.436992
6,seg_004cd2,8.953513
7,seg_004ee5,5.676419
8,seg_004f1f,5.461224
9,seg_00648a,3.137615


In [11]:
out_df.to_csv('lgbm_results.csv',index=False)